<a href="https://colab.research.google.com/github/gunisha30/projects/blob/master/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords

df = pd.read_csv("//content//drive//My Drive//spam.csv")

#frequency of ham and spam 
pd.crosstab(df['Label'],columns='freq')

#stemming
ps = nltk.PorterStemmer()

#list of stopwords
stopwords = nltk.corpus.stopwords.words('english')

#returns percentage of punctuations in text
def punct_pc(text):
    punct_count = sum([1 for char in text if char in string.punctuation])
    return (punct_count/(len(text) - text.count(' ')))*100

#creating two new features
df['text_length'] = df['EmailText'].apply(lambda x : len(x)-x.count(' '))
df['punct'] = df['EmailText'].apply(lambda x : punct_pc(x))

#remove punctuations
def clean_data(text):
    punct = "".join([word.lower() for word in text if word not in string.punctuation])
    splt = re.split('\W+',punct)
    txt = [ps.stem(word) for word in splt if word not in stopwords]
    return txt

x_train,x_test,y_train,y_test = train_test_split(df[['EmailText','text_length','punct']],df['Label'],test_size=0.2,random_state=123)

#vectorising using count vectorizer
vect=CountVectorizer(analyzer=clean_data)

#fit- done on training data, tranform- applied to training and testing data
vectfit = vect.fit(x_train['EmailText'])
xtrainvect = vectfit.transform(x_train['EmailText'])
xtestvect = vectfit.transform(x_test['EmailText'])
xtrainvect= pd.concat([x_train[['text_length','punct']].reset_index(drop=True),pd.DataFrame(xtrainvect.toarray())],axis=1)
xtestvect=pd.concat([x_test[['text_length','punct']].reset_index(drop=True),pd.DataFrame(xtestvect.toarray())],axis=1)

#classifier- random forest
rf = RandomForestClassifier(random_state=123,n_jobs=1)

#get the list of hyperparameters, n_estimators=num of trees in the forest, max_features= num of features considered for splitting
#print(rf.get_params())

param = {'n_estimators' : [10,25,50,100,300], 'max_depth' : [10, 20, 50,100, None],'max_features' : [10,50,'auto']}
grid = GridSearchCV(rf,param,cv=5,n_jobs=4)

rf_grid_fit_2 = grid.fit(xtrainvect, y_train)
#pd.DataFrame(rf_grid_fit_2.cv_results_).sort_values('mean_test_score',ascending=False)[0:10]

#y_train.head()

In [ ]:
!pip install nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install pyPDF2

In [ ]:
import PyPDF2
pdfFileObj = open('Gunisharesume.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
#print(pdfReader.numPages)
pageObj = pdfReader.getPage(0) 
print(pageObj.extractText()) 
pdfFileObj.close()  